In [ ]:
# OLD CODE --- IGNORE ---
def download_file(url, download_dir=DOWNLOAD_DIR):
    os.makedirs(download_dir, exist_ok=True)
    file_name = os.path.join(download_dir, os.path.basename(url))
    print(file_name)
    subprocess.run(["wget", "-q", "-N", "-P", download_dir, url], check=True)
    subprocess.run(["unzip", "-o", file_name, "-d", download_dir], check=True)
    unzip_dir = Path(f"{download_dir}/unzipped_files") # will use f"{file_name.strip('.zip.csv')}" in DAG to create cloud directory 
    archive_dir = Path(f"{download_dir}/archive_files") # TODO: remember use Pthlib or os.path to join paths for prod
    # create directories to store unzipped and archived files
    subprocess.run(["mkdir", "-p", unzip_dir], check=True)
    subprocess.run(["mkdir", "-p", archive_dir], check=True)
    # move unzipped and archived files to respective directories
    unzipped_file = file_name.strip('.zip')
    archive = file_name
    subprocess.run(["mv", unzipped_file, unzip_dir], check=True)
    subprocess.run(["mv", archive, archive_dir], check=True)
    return file_name


    def upload_data(**kwargs):
    backend = os.environ.get("STORAGE_BACKEND")

    if backend == "gcp":
        from google.cloud import storage
        client = storage.Client()
        bucket = client.bucket(os.environ["GCP_BUCKET_NAME"])
        blob = bucket.blob("data/citi_bike.csv")
        blob.upload_from_filename("/tmp/citi_bike.csv")

    elif backend == "azure":
        from azure.storage.blob import BlobServiceClient
        blob_service = BlobServiceClient.from_connection_string(os.environ["AZURE_CONN_STR"])
        blob_client = blob_service.get_blob_client(container=os.environ["AZURE_CONTAINER_NAME"],
                                                   blob="data/citi_bike.csv")
        with open("/tmp/citi_bike.csv", "rb") as data:
            blob_client.upload_blob(data, overwrite=True)


@safe_run
def download_files_wl(url, download_dir=DOWNLOAD_DIR):        
    filecount = 0
    # create directories to store unzipped and archived files

    os.makedirs(download_dir, exist_ok=True)
    archive_dir = Path(f"{download_dir}/archive_files") 
    file_path = Path(archive_dir) / os.path.basename(url)
    files_dir = str(os.path.basename(url)).strip('JC-citibike-tripdata.zip.csv')
    unzip_dir = Path(f"{download_dir}/unzipped_files/{files_dir}")

    # Download using subprocess and wget
    #print(f"Downloading {url} to {file_path}")
    subprocess.run(["wget", "-q", "-N", "-P", archive_dir, url], check=True)

    # Extract depending on file type
    if file_path.suffix == ".zip":
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
        #print(f"Extracted ZIP to {unzip_dir}")
    elif file_path.suffix in [".tar", ".gz", ".bz2"]:
        with tarfile.open(file_path, 'r:*') as tar_ref:
            tar_ref.extractall(unzip_dir)
        #print(f"Extracted TAR to {unzip_dir}")
    else:
        print("No extraction performed, unknown file type")

    
    filecount += 1
    print(f"Download and extraction of {os.path.basename(url)} complete. Total files processed: {filecount}")

for url in files_list[-1:]:
    download_files_wl(url)


    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".asm") or filename.endswith(".py"): 
        # print(os.path.join(directory, filename))
        continue
    else:
        continue

while True:
        start_time = time()
        
        df = next(df_iter)
        df.to_sql(name="citibike_data", con=engine, if_exists="append")

        end_time = time()

        print(f'inserted another chunk, ops took {(end_time-start_time):.2f}')


# Create an iterator from the large dataset
df_iter = pd.read_csv(path, iterator=True, chunksize=100000, parse_dates=["started_at", "ended_at"])
df = next(df_iter)

# Create schema in psql database
df.head(n=0).to_sql(name="citibike_data_202508", con=engine, if_exists="replace")

In [ ]:
citibikebq_engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}', 
                       isolation_level='AUTOCOMMIT')

try:
    with citibikebq_engine.connect() as citibikebq_conn:
        # Initialize BigQuery client
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/bonaventure/gcp-keys.json"

        client = bigquery.Client()

        # Download data from BigQuery and load to Postgres
        for year in range(2013, 2015):  # TODO: Extend range until year 2019 in production or cloud environment
            #print(year)
            # Query BigQuery in 1 million row chunks
            chunk_size = 1000000  # 1 million rows per chunk
            offset = 0

            while True:
                try:
                    query = f"""
                    SELECT *
                    FROM `bigquery-public-data.new_york_citibike.citibike_trips`
                    WHERE EXTRACT(YEAR FROM starttime) = {year}
                    LIMIT {chunk_size} OFFSET {offset}
                    """
                    
                    df_chunk = client.query(query).to_dataframe()
                    if df_chunk.empty:
                        logging.info(f"Insertion into postgres db '{DB_NAME}' complete: %s", "citibike_trips_{year}")
                        break  # stop when there is no more data
                    
                    df_chunk.to_sql(f'citibike_trips_{year}', engine, if_exists='replace', index=False)
                    logging.info(f"Loaded rows {offset} to {offset + len(df_chunk)} into Postgres")
                    #print(f"Loaded rows {offset} to {offset + len(df_chunk)} into Postgres")

                    offset += chunk_size
                except StopIteration:
                    logging.info(f"Insertion into postgres db {DB_NAME} complete: citibike_trips_{year}; just check if last")
                    break
except Exception as e:
            logging.error("Data insertion from Big Query failed: %s", e)
            raise 



In [ ]:
        def ingest_from_bigquery_to_postgres(params=params, chunk_size=1000_000):
            """Ingest data from Big Query to Postgres in chunks"""

            # Replace these with your PostgreSQL credentials
            DB_USER = params.user
            DB_PASS = params.password
            DB_HOST = params.host
            DB_PORT = params.port
            DB_NAME = 'citibikebq'  # The database you want to create

            # Connect to default database and create new database if not exists
            default_engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/postgres', 
                                isolation_level='AUTOCOMMIT')
            try:
                # Execute CREATE DATABASE
                with default_engine.connect() as default_conn:
                    # Check if the database exists
                    result = default_conn.execute(text("SELECT 1 FROM pg_database WHERE datname = :dbname"), {"dbname": DB_NAME})
                    exists = result.scalar()  # Returns None if no rows found
                    
                    if not exists:
                        default_conn.execute(text(f"CREATE DATABASE {DB_NAME}"))
                        #print(f"Database '{DB_NAME}' created successfully!")
                        logging.info(f"Database '{DB_NAME}' created successfully!")
                    else:
                        #print(f"Database '{DB_NAME}' already exists.")
                        logging.info(f"Database '{DB_NAME}' already exists.")
            except Exception as e:
                        logging.error("Data insertion from Big Query failed: %s", e)
                        raise 
            
            # Connect to the newly created database and ingest data into postgres container
            citibikebq_engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}', 
                            isolation_level='AUTOCOMMIT')

            try:
                with citibikebq_engine.connect() as citibikebq_conn:
                    # Initialize BigQuery client
                    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/bonaventure/gcp-keys.json"

                    client = bigquery.Client()

                    # Download data from BigQuery and load to Postgres
                    for year in range(2013, 2015):  # TODO: Extend range until year 2019 in production or cloud environment
                        #print(year)
                        # Query BigQuery in 1 million row chunks
                        chunk_size = chunk_size  # 1 million rows per chunk
                        offset = 0

                        while True:
                            try:
                                query = f"""
                                SELECT *
                                FROM `bigquery-public-data.new_york_citibike.citibike_trips`
                                WHERE EXTRACT(YEAR FROM starttime) = {year}
                                LIMIT {chunk_size} OFFSET {offset}
                                """
                                
                                df_chunk = client.query(query).to_dataframe()
                                if df_chunk.empty:
                                    logging.info(f"Insertion into postgres db '{DB_NAME}' complete: %s", "citibike_trips_{year}")
                                    break  # stop when there is no more data
                                
                                df_chunk.to_sql(f'citibike_trips_{year}', engine, if_exists='replace', index=False)
                                logging.info(f"Loaded rows {offset} to {offset + len(df_chunk)} into Postgres")
                                #print(f"Loaded rows {offset} to {offset + len(df_chunk)} into Postgres")

                                offset += chunk_size
                            except StopIteration:
                                logging.info(f"Insertion into postgres db {DB_NAME} complete: citibike_trips_{year}; just check if last")
                                break
            except Exception as e:
                        logging.error("Data insertion from Big Query failed: %s", e)
                        raise 